# 第二节 理解模型IO (模型、提示、输出)

### 环境安装配置

将导入OS，导入OpenAI、langchain-openai，并加载OpenAI秘钥。

In [ ]:
%pip install --upgrade --quiet openai langchain-openai langchain

In [ ]:
import os
API_SECRET_KEY = "填写【个人资料】中获取的最新token"
BASE_URL = "https://www.ai360labs.com/openai/v1/"

os.environ["OPENAI_API_KEY"] = API_SECRET_KEY
os.environ["OPENAI_BASE_URL"] = BASE_URL

In [ ]:
import openai
from langchain_openai import ChatOpenAI,OpenAI

#### OpenAI原生调用

OpenAI原生的chat API 被视为行业的标准调用方式。其他模型厂商的API接口都跟OpenAI的调用方式雷同。学习了OpenAI的方式，等同于学习了整个行业的标准。


定义一个辅助函数 `get_completion`，你可以使用`get completion`来询问1加1等于什么，这将调用ChatGPT，或技术上说是模型GPT 3.5 Turbo，来给你回答。

In [ ]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.chat.completions.create(model=model,messages=messages)
    return response.choices[0].message.content

输入你的问题。

In [ ]:
print(get_completion("中国的首都是哪里?"))

北京

为了激发对LangChain模型、提示和解析器抽象的兴趣，假设你需要一个翻译助理，帮你翻译客户发来的邮件。

提示词可以使用模板字符串，包含变量，比如要设置不同的说话口吻，选择不同的邮件内容，就将口吻和邮件内容设置为一个变量。最后的prompt使用f-string拼接一体。例如在线代码的`{style}`和`{customer_email}`。

In [ ]:
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse,\
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

为了解决这个问题，我们将要求这个LLM将邮件翻译中文，并保持冷静和尊重的语气。

In [ ]:
style = "保持冷静和尊重的语气"

请你尝试尝试修改提示，看看你是否能得到不同的输出。修改风格或者邮件内容，指导大型语言模型完成翻译任务。

In [ ]:
prompt = f"""我不懂英语，现在请你为我翻译客户的邮件为中文。 \
翻译的风格是：{style}。\
需要翻译的内容是: ```{customer_email}```
"""

print(prompt)

In [ ]:
response = get_completion(prompt)

In [ ]:
response

如果你有不同的客户以不同的语言写评论，不仅仅是英语，还有法语、德语、日语等，你可以想象需要生成一系列提示来生成这样的翻译。

#### LangChain封装调用

现在我们看看LangChain又是如何达成这个目标的。还记得创始人为什么发明LangChain吗？在LLM的编程实践中，AI开发者需要很多胶水代码。LangChain做了很多这样的抽象，简化AI开发者的编程。

LangChain的设计目标之一是简化AI开发者的工作流程，减少对不同模型厂商API的依赖和学习成本。

使用LangChain后，我们无需封装每个模型厂商的API函数，也无需查看每个模型厂商的API格式。

LangChain提供了一套统一的接口和抽象，使得开发者可以更方便地使用不同模型厂商的模型。

通过LangChain，我们可以使用相同的方式调用不同模型厂商的API，无论是OpenAI、GPT-3.5 Turbo还是其他模型。

这种统一的接口和抽象使得开发者可以更专注于业务逻辑的实现，而无需关注底层模型的具体实现细节。

因此，LangChain大大简化了AI开发者的工作流程，提高了开发效率和代码可维护性。


LangChain v0.1.0 版本之后，调用的方式如下：

**注意 langchain_openai 是下划线** ,本教程使用的是LangChain v0.1.0 版本。

In [ ]:
from langchain_openai import ChatOpenAI

当构建应用程序时，通常是将温度参数设置为0。默认温度是0.7。温度设置为0，使这个输出稳定一点，虽然会散失一些创造力。

In [ ]:

chat = ChatOpenAI(temperature=0.0)
result = chat.predict("中国的首都是哪里?")
result


返回：'中国的首都是北京。'

## 提示词模板

LangChain有专门的langchain.prompts模块，处理提示词的构建任务。现在以LangChain的方式构建一个提示词，用于对话式要LLM翻译客户邮件内容。

定义模板字符串如下：

In [ ]:
template_string = """Translate the text \
that is delimited by triple backticks \
into a style that is {style}. \
text: ```{text}```
"""

首先导入ChatPromptTemplate，用于构建符合chat类型API的提示词模板。将f-string模板字符串设置为变量template_string，ChatPromptTemplate接受该参数，接着实例化为一个对象prompt_template。

In [ ]:
from langchain.prompts import ChatPromptTemplate
prompt_template = ChatPromptTemplate.from_template(template_string)

我们查看这个对象所有的属性，更加直观感受LangChain封装的内容。

这里我们思考一下，为什么LangChain要做提示词模板的封装。

事实上，如果我打印出来，它意识到它有两个输入变量，风格和文本。

In [ ]:
prompt_template.messages[0].prompt

我们之前注入了`style`和`text`两个变量。通过这样的方式可以找到这两个占位符名称：

回答上面的思考题，经过LangChain的封装，我们不仅减少了格式转换问题，而且对提示词的内容可以进行动态管理（增删改查操作）。

当你构建复杂的LLM应用时，提示词模板内部完全可能是变化的。

比如你的程序需要做个性化，可用之前的模板，但要添加一点不一样的内容，此时提示词模板可以像数据一样可以管理，而不是重写。

In [ ]:
prompt_template.messages[0].prompt.input_variables

占位符`style`设置为变量`customer_style`

In [ ]:
customer_style = "严谨"

占位符`text`设置为变量`customer_email`

In [ ]:
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse, \
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

创建客户消息，这将生成提示，并将此大型语言模型传递给稍后以获得回应。客户消息实际上是一个列表。

调用format_messages方法，将提示词模板字符串转换为chat类型所要求的消息列表格式。消息列表格式包含了不同角色，在原生OpenAI的调用中，我们指定了OpenAI system,user,assistant 角色。
如果不转换为消息列表格式是无法调用LLM chat类型的接口。
```
# 原生OpenAI的调用方法
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}] #这里还可以新增其他角色
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, 
    )
    return response.choices[0].message["content"]
```    

本质上来说是LangChain 的胶水代码将这些转换工作承包完了，开发者可以不用做这类型的转换工作。

In [ ]:
customer_messages = prompt_template.format_messages(
                    style=customer_style,
                    text=customer_email)

customer_messages 是一个列表，我们现在查看这个消息是不是一个列表。

In [ ]:
print(type(customer_messages))

看看列表的第一个元素，就是你期望它创建的提示。

In [ ]:
print(type(customer_messages[0]))

最后，让我们将这个提示传递给LLM，调用chat，作为对OpenAI chatGPT端点的引用。

In [ ]:
# Call the LLM to translate to the style of the customer message
# Reference: chat = ChatOpenAI(temperature=0.0)
chat = ChatOpenAI(temperature=0.0)
customer_response = chat(customer_messages)

如果我们打印出客户响应的内容，那么它就会把这个文本从英语海盗翻译成严谨的中文。

In [ ]:
print(customer_response.content)

我们再试试翻译为“甜妹”语气。

In [ ]:
service_style = """\
温柔的甜美女生风格 \
"""

将提示词模板字符串转换为消息列表格式。

重新使用该提示模板，并指定我们希望的输出风格是温柔的甜美女生风格，文本是英文邮件。

In [ ]:
service_messages = prompt_template.format_messages(
    style=service_style,
    text=customer_email)

print(service_messages[0].content)

对比两种语气翻译结果的区别。

In [ ]:
service_response = chat(service_messages)
print(service_response.content)

这段代码展示了提示词模板的作用，它可以帮助我们自由组织和重新构造提示词。

在大语言模型开发中，提示词的构造是一个重要的特性。好的提示词不仅可以产生好的输出，还可以提高工程化水平。

通过使用提示词模板，我们可以更好地利用大语言模型的能力，提供更准确、多样化的提示词。


## 解析器

解析器的重要性在于将这些非结构化的文本转换为结构化的数据形式，以便进一步分析和利用。

结构化数据更容易被计算机程序处理，可以更方便地进行存储、检索和分析，从而提高了数据的可用性和应用价值。

提示词模板用于指定LLM的输入，解析器来正确解释LLM给出的输出，这样共同构成了一种很好的抽象，完整控制了模型的输入、输出。



让我们看看使用LangChain的输出解析器的示例。示例将是从产品评论中提取信息，并以JSON格式格式化输出。

假设我们现在想通过LLM程序，在对话中提取一个JSON字符串，解析这个JSON后获得数据中的关键信息。

任务是要求LLM将客户评论作为输入并提取这三个字段，然后将输出格式化为JSON，具有以三个键。

In [ ]:
{
  "gift": False,
  "delivery_days": 5,
  "price_value": "pretty affordable!"
}



使用一个电商的好评数据：

In [ ]:
customer_review = """\
This leaf blower is pretty amazing.  It has four settings:\
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.
"""

构建一个好评提示模板，告知LLM 所需要的字段释义，要求LLM回答时，使用指定键值且以JSON格式回答问题。

In [ ]:
review_template = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product \
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

Format the output as JSON with the following keys:
gift
delivery_days
price_value

text: {text}
"""

导入LangChain的提示词模块 `langchain.prompts`。将此类提示包装在LangChain提示词模块中，使重用此类提示变得更容易。

LangChain还为一些常见操作提供提示，例如摘要、问答、连接SQL数据库或连接不同的API。

因此，通过使用LangChain的内置提示，你可以快速让应用程序运行，而无需自己设计提示。

LangChain提示词模板的另一个作用是它还支持**输出解析**。

In [ ]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(review_template)
print(prompt_template)

开始运行，运行代码跟之前一样，转换消息列表格式后，发起chat。


创建消息列表的提示模板，传递给OpenAI端点，然后让我们打印出响应。

In [ ]:
messages = prompt_template.format_messages(text=customer_review)
chat = ChatOpenAI(temperature=0.0)
response = chat(messages)

打印结果,`print(type(response.content))`是个json字符串：

In [ ]:
print(response.content)

尝试获取gift字段的内容。

运行代码后，你看到了一个类型错误。

In [ ]:
# You will get an error by running this line of code 
# because'gift' is not a dictionary
# 'gift' is a string
response.content.get('gift')

这实际上是一个字符串，不是一个Python字典。所以无法直接获取到gift字段的值。

In [ ]:
print(type(response.content))


所以，让我们看看如何使用LangChain的解析器来解决这个错误。

### LangChain解析器应用示例

现在我们开始跟上面提示词指导模型输出不一样的旅程，使用LangChain解析器用于从产品评论中提取产品特征或用户反馈。

解析器可以从评论文本中提取产品特性、用户体验评价或情感倾向等信息，并将其转换为结构化的数据形式，以便进行分析和报告。

首先，从LangChain中导入响应模式 ResponseSchema 和结构化输出解析器 StructuredOutputParser。

In [ ]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

指定所需要提取的字段名称和描述。

告诉它通过指定这些响应模式来解析。所以礼物模式名为礼物，有礼物的描述。购买的商品是否为他人礼物？回答真或是，假如果不是或未知，等等。

还有一个礼物模式，送货日期模式。

In [ ]:
gift_schema = ResponseSchema(name="gift",
                             description="Was the item purchased\
                             as a gift for someone else? \
                             Answer True if yes,\
                             False if not or unknown.")
delivery_days_schema = ResponseSchema(name="delivery_days",
                                      description="How many days\
                                      did it take for the product\
                                      to arrive? If this \
                                      information is not found,\
                                      output -1.")
price_value_schema = ResponseSchema(name="price_value",
                                    description="Extract any\
                                    sentences about the value or \
                                    price, and output them as a \
                                    comma separated Python list.")

response_schemas = [gift_schema, 
                    delivery_days_schema,
                    price_value_schema]

## 使用LangChain的输出解析器。

目前为止，我们已经为这些指定了模式。

实际上，LangChain已经给了LLM如何输出的提示词，你希望发送给LLM的指令已经包含在LangChain内置的提示模板了。

我们按照以下步骤运用一个输出解析器。

- 定义响应模式，创建一个StructuredOutputParser类的实例

- 定义消息列表

- 调用LLM获取响应

- 解析响应

- 构建JSON格式输出

- 打印输出结果



使用from_response_schemas方法，创建一个StructuredOutputParser类的实例。
- 这个方法接受response_schemas参数，它是一个包含多个schema的列表。
- output_parser变量被赋值为创建的StructuredOutputParser类的实例。



In [ ]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

打印输出解析器的内容。最后面我们可以看到LangChain 封装的提示词。

```output them as a comma separated Python list.```

In [ ]:
print(output_parser)

我们还可以查看，最终给LLM发送的指导词。LangChain有一套相当精确的指令供LLM使用，这将使其生成输出解析器可以处理的输出。

将三个字段名的数据模式注入到输出解析器后，我们便可以使用输出解析器管理提示词。

首先调用输出解析器的get_format_instructions方法，生产出`{format_instructions}`字符串，它包含了:

- 三个字段数据模式。 
 ```json
{
	"gift": string  // Was the item purchased                             as a gift for someone else?                              Answer True if yes,                             False if not or unknown.
	"delivery_days": string  // How many days                                      did it take for the product                                      to arrive? If this                                       information is not found,                                      output -1.
	"price_value": string  // Extract any                                    sentences about the value or                                     price, and output them as a                                     comma separated Python list.
}
```

- 要求LLM输出的结果为json格式的指令。

 (The output should be a markdown code snippet formatted in the following schema, including the leading and trailing )

In [ ]:
format_instructions = output_parser.get_format_instructions()

In [ ]:
print(format_instructions)

接着，我们删除了在原始提示词 review_template 内要求LLM输出的结果为json格式的指令，提示词底部添加刚刚生产的{format_instructions}，保存为 review_template_2。

In [ ]:
# 差别在删除了LLM输出为指定键值的JSON格式的指示词。
review_template_2 = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product\
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

text: {text}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template=review_template_2)

messages = prompt.format_messages(text=customer_review, 
                                format_instructions=format_instructions)

查看完整的提示词。

In [ ]:
print(messages[0].content)

查看最终的结果。

In [ ]:
response = chat(messages)

In [ ]:
print(response.content)

运用parse方法，将你的LLM输出解析为Python字典，使输出更容易在下游处理中使用。

In [ ]:
output_dict = output_parser.parse(response.content)

In [ ]:
output_dict

打印后，确认是dict类型。

In [ ]:
type(output_dict)

至此我们将报错的代码修复好了，再次尝试使用get方法获取gift的值。

In [ ]:
output_dict.get('gift')